In [1]:
import pandas as pd
import numpy as np
import os
import arrow

#### 1.数据导入

In [6]:
file_path = r'D:\rui\code_analysis\课程资料\6. 综合专题二：基于高铁余票的客流行为特征及其效应分析\12306data'
file = os.path.join(file_path,'Ninghu-2016-12-21.csv')

In [7]:
df_Ninghu = pd.read_csv(file)

In [10]:
df_Ninghu.head()

,54000G702940,G7029,南京,镇江,05:49,0,0.1,94,993,0.2,2016-12-21,00:00
0,54000G703350,G7033,南京,镇江,06:06,0,0,43,493,160,2016-12-21,00:00
1,54000G710140,G7101,南京,镇江,06:11,0,0,46,509,164,2016-12-21,00:00
2,54000G703530,G7035,南京,镇江,06:31,0,0,48,493,167,2016-12-21,00:00
3,5l000G758100,G7581,南京南,镇江,06:45,10,0,24,417,147,2016-12-21,00:00
4,5l000G750550,G7505,南京南,镇江南,07:13,25,0,92,432,260,2016-12-21,00:00


#### 2.数据整理

In [16]:
#添加表头
df_Ninghu = pd.read_csv(file,names=['编号', '车次', '发车站', '到达站', '发车时间', '商务特等座',
                                    '动卧', '一等座', '二等座','无座','记录日期','记录时间'])

In [17]:
df_Ninghu.tail()

,编号,车次,发车站,到达站,发车时间,商务特等座,动卧,一等座,二等座,无座,记录日期,记录时间
2445513,240000G1570H,G157,无锡东,上海虹桥,22:54,8,0,63,657,0,2016-12-21,23:03
2445514,240000G1570H,G157,无锡东,上海虹桥,22:54,8,0,63,657,0,2016-12-21,23:05
2445515,4f000G194200,G1939,苏州北,上海虹桥,22:52,10,0,22,428,0,2016-12-21,23:05
2445516,240000G1570H,G157,无锡东,上海虹桥,22:54,8,0,63,657,0,2016-12-21,23:06
2445517,240000G1570H,G157,无锡东,上海虹桥,22:54,8,0,63,657,0,2016-12-21,23:07


In [36]:
#用正则表达式过滤错误时间的信息
df_Ninghu = df_Ninghu[df_Ninghu['发车时间'].str.contains('0[3-9]:|1[0-9]:|2[0-3]:')]

In [61]:
#arrow时间推移
arrow.get('have a time 2020-06-21 ,maybe you can extract date in it','YYYY-MM-DD').shift(days=+100)

<Arrow [2020-09-29T00:00:00+00:00]>

In [70]:
df_Ninghu['停售时间'] = df_Ninghu['发车时间'].apply(lambda x:arrow.get(x,'HH:mm').shift(minutes=-30).format('HH:mm'))

In [71]:
df_Ninghu.head()

,编号,车次,发车站,到达站,发车时间,商务特等座,动卧,一等座,二等座,无座,记录日期,记录时间,停售时间
0,54000G702940,G7029,南京,镇江,05:49,0,0,94,993,0,2016-12-21,00:00,05:19
1,54000G703350,G7033,南京,镇江,06:06,0,0,43,493,160,2016-12-21,00:00,05:36
2,54000G710140,G7101,南京,镇江,06:11,0,0,46,509,164,2016-12-21,00:00,05:41
3,54000G703530,G7035,南京,镇江,06:31,0,0,48,493,167,2016-12-21,00:00,06:01
4,5l000G758100,G7581,南京南,镇江,06:45,10,0,24,417,147,2016-12-21,00:00,06:15


In [72]:
#过滤出在停售时间的记录
df_Ninghu_use = df_Ninghu[df_Ninghu['记录时间'] == df_Ninghu['停售时间']]

In [74]:
len(df_Ninghu_use)

2785

In [75]:
#新增总余票量
df_Ninghu_use['总票数'] = df_Ninghu_use['商务特等座'] + df_Ninghu_use['动卧'] +\
df_Ninghu_use['一等座'] + df_Ninghu_use['二等座'] + df_Ninghu_use['无座']

C:\Users\cc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
df_Ninghu_use.to_csv(r'../../file/subject/df_Ninghu_use.csv')

In [77]:
os.getcwd()

'D:\\rui\\code_analysis\\homework\\专题研究'